In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
dataset = pd.read_csv('../input/robi-datathon-2-pre-assessment/train.csv')
dataset.drop(dataset.columns[[0,5,6,7,9,10,11,12,13,14,16,17,18,19]], axis = 1, inplace = True)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
#gender
X[:, 0] = le.fit_transform(X[:, 2])
#s11
X[:, 1] = le.fit_transform(X[:, 2])
#s12
X[:, 2] = le.fit_transform(X[:, 2])
#s58
X[:, 5] = le.fit_transform(X[:, 2])

In [26]:
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.3, random_state = 0)


In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
XTrain = sc.fit_transform(XTrain)
XTest = sc.transform(XTest)

In [28]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'] )

In [29]:
ann.fit(XTrain, yTrain, batch_size=32, epochs=100)

In [24]:
y_pred = ann.predict(XTest)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), yTest.reshape(len(yTest), 1)), 1))

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(yTest, y_pred)
print(cm)
accuracy_score(yTest, y_pred)